## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [13]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Activation 

In [27]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


import keras
num_classes = 10
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [28]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [29]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3), 
                                 activation='relu',
                                 input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

# 池化
classifier.add(MaxPooling2D(pool_size=(2, 2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

classifier.summary()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 13, 13, 32)        9248      
_________________________________________________________________
batch_normalization_12 (Batc (None, 13, 13, 32)        128       
_________________________________________________________________
flatten_5 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               540900    
__________

In [31]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=200,epochs=10)

Epoch 1/10
50000/50000 [==============================] - 451s 9ms/step - loss: 1.2961 - acc: 0.5622
Epoch 2/10
50000/50000 [==============================] - 381s 8ms/step - loss: 0.8668 - acc: 0.6991
Epoch 3/10
50000/50000 [==============================] - 387s 8ms/step - loss: 0.6727 - acc: 0.7676
Epoch 4/10
50000/50000 [==============================] - 383s 8ms/step - loss: 0.5260 - acc: 0.8189
Epoch 5/10
50000/50000 [==============================] - 380s 8ms/step - loss: 0.3968 - acc: 0.8651
Epoch 6/10
50000/50000 [==============================] - 387s 8ms/step - loss: 0.2978 - acc: 0.8993
Epoch 7/10
50000/50000 [==============================] - 388s 8ms/step - loss: 0.2186 - acc: 0.9258
Epoch 8/10
50000/50000 [==============================] - 385s 8ms/step - loss: 0.1585 - acc: 0.9476
Epoch 9/10
50000/50000 [==============================] - 394s 8ms/step - loss: 0.1253 - acc: 0.9601
Epoch 10/10
50000/50000 [==============================] - 384s 8ms/step - loss: 0.1126 - a

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [32]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[7.6246512e-01, 1.7337849e-06, 2.8874664e-02, 2.0639518e-01,
        5.2869919e-04, 2.4629297e-04, 2.0936529e-04, 1.1552599e-03,
        1.2355309e-04, 1.0837685e-07]], dtype=float32)